In [53]:
from typing import List, Tuple
import random
import numpy as np
import stim

## Generating errors

Here we'll generate errors using the uniform depolarizing channel:
$D[\rho] = (1-\lambda) \rho + \frac{\lambda}{3} (X \rho X + Y \rho Y + Z \rho Z)$,
where $0 \leq \lambda \leq 1$.

In [32]:
# We will represent Pauli strings initially as a list of n integers where n_i in {0, 1, 2, 3}
# 0 -> I
# 1 -> X
# 2 -> Y
# 3 -> Z
# e.g. XIYZ = [1, 0, 2, 3]

random.seed(123)

def random_depolarizing_error(nqubits: int, lamb: float) -> List[int]:
    """Generate a random error depolarizing error on n qubits."""

    pauli = [0] * nqubits
    for i in range(len(pauli)):
        r = random.random()
        if r <= lamb:
            # Identity
            pauli[i] = 0
        else:
            # Generate an X, Y, or Z error with equal probabilities.
            r2 = random.random()
            if r2 <= 1. / 3.:
                # X error
                pauli[i] = 1
            elif r2 >= 1. / 3. and r2 <= 2. / 3.:
                # Y error
                pauli[i] = 2
            else:
                # Z error
                pauli[i] = 3
    return pauli

In [41]:
pauli = random_depolarizing_error(3, 0.5)
print(pauli)

[2, 1, 0]


In [69]:
# Next convert to binary symplectic form.

def pauli_to_binary_symplectic(pauli: List[int]) -> np.ndarray:
    """Convert a Pauli string represented by integers to its binary form."""

    bin_pauli = [False] * (2 * len(pauli))
    for i, p in enumerate(pauli):
        assert p in set([0, 1, 2, 3]), f"Pauli {p} at index {i} is is invalid (should be in 0,1,2,3)"
        if p == 1 or p == 2:
            bin_pauli[i] = True
        if p == 2 or p == 3:
            bin_pauli[i + len(pauli)] = True
    return np.array(bin_pauli)


def binary_symplectic_to_pauli(bin_pauli: np.ndarray) -> List[int]:
    """Convert the binary-symplectic form to a list of integers."""

    assert bin_pauli.size % 2 == 0
    nq = bin_pauli.size // 2
    x_bits = bin_pauli[:nq]
    z_bits = bin_pauli[nq:]
    pauli = [0] * nq
    for i, (xbit, zbit) in enumerate(zip(x_bits, z_bits)):
        if xbit and not zbit:
            pauli[i] = 1
        elif xbit and zbit:
            pauli[i] = 2
        elif not xbit and zbit:
            pauli[i] = 3
        else:
            pauli[i] = 0
    return pauli

In [50]:
pauli_to_binary_symplectic([0, 1, 3, 2])

array([False,  True, False,  True, False, False,  True,  True])

## Majority rules decoding for Shor's nine-qubit code

## Lookup table decoder for the Steane code

See https://pennylane.ai/qml/demos/tutorial_bp_catalyst

In [52]:
# A CSS code has two check matrices: one for X errors and another for Z errors.

H_x = np.array(
    [
        [False, False, False, True, True, True, True],
        [False, True, True, False, False, True, True],
        [True, False, True, False, True, False, True]
    ]
)
H_z = H_x.copy()

In [54]:
def error_to_steane_syndrome(err: np.ndarray) -> np.ndarray:
    """Convert a (binary-symmetric) error on 7 qubits to its Steane syndrome."""

    assert err.size == 14
    x_err = err[:7]
    z_err = err[7:]
    x_syndrome = H_x @ x_err
    z_syndrome = H_z @ z_err
    return np.concat([x_syndrome, z_syndrome], axis=0)

In [66]:
err = pauli_to_binary_symplectic([0, 0, 0, 2, 0, 0, 0])
syndrome = error_to_steane_syndrome(err)
print(syndrome)

[ True False False  True False False]


In [77]:
# Here students will notice the pattern of syndromes for all single-qubit errors.

for i in range(7):
    for p in [1, 2, 3]:
        err_pauli = [0] * 7
        err_pauli[i] = p
        err_binary = pauli_to_binary_symplectic(err_pauli)
        syndrome = error_to_steane_syndrome(err_binary)
        print(err_pauli, syndrome)

[1, 0, 0, 0, 0, 0, 0] [False False  True False False False]
[2, 0, 0, 0, 0, 0, 0] [False False  True False False  True]
[3, 0, 0, 0, 0, 0, 0] [False False False False False  True]
[0, 1, 0, 0, 0, 0, 0] [False  True False False False False]
[0, 2, 0, 0, 0, 0, 0] [False  True False False  True False]
[0, 3, 0, 0, 0, 0, 0] [False False False False  True False]
[0, 0, 1, 0, 0, 0, 0] [False  True  True False False False]
[0, 0, 2, 0, 0, 0, 0] [False  True  True False  True  True]
[0, 0, 3, 0, 0, 0, 0] [False False False False  True  True]
[0, 0, 0, 1, 0, 0, 0] [ True False False False False False]
[0, 0, 0, 2, 0, 0, 0] [ True False False  True False False]
[0, 0, 0, 3, 0, 0, 0] [False False False  True False False]
[0, 0, 0, 0, 1, 0, 0] [ True False  True False False False]
[0, 0, 0, 0, 2, 0, 0] [ True False  True  True False  True]
[0, 0, 0, 0, 3, 0, 0] [False False False  True False  True]
[0, 0, 0, 0, 0, 1, 0] [ True  True False False False False]
[0, 0, 0, 0, 0, 2, 0] [ True  True False

In [74]:
# Ideally, students should derive this themselves!
# Possibly this can be done by just looking at the syndromes for each single-qubit error,
# and noticing that they make all the octal digits.

def steane_lookup_decoder(syndrome: np.ndarray) -> np.ndarray:
    """Maps Steane syndromes onto their minimum-weight corrections."""

    def decode_x_or_z(syndrome):
        if np.all(syndrome == np.array([False, False, False])):
            return np.array([False, False, False, False, False, False, False])
        elif np.all(syndrome == np.array([False, False, True])):
            return np.array([True, False, False, False, False, False, False])
        elif np.all(syndrome == np.array([False, True, False])):
            return np.array([False, True, False, False, False, False, False])
        elif np.all(syndrome == np.array([False, True, True])):
            return np.array([False, False, True, False, False, False, False])
        elif np.all(syndrome == np.array([True, False, False])):
            return np.array([False, False, False, True, False, False, False])
        elif np.all(syndrome == np.array([True, False, True])):
            return np.array([False, False, False, False, True, False, False])
        elif np.all(syndrome == np.array([True, True, False])):
            return np.array([False, False, False, False, False, True, False])
        elif np.all(syndrome == np.array([True, True, True])):
            return np.array([False, False, False, False, False, False, True])
        else:
            raise ValueError(f"Syndrome {syndrome} not valid.")

    assert syndrome.size == 6
    x_syndrome = syndrome[:3]
    z_syndrome = syndrome[3:]
    x_correction = decode_x_or_z(x_syndrome)
    z_correction = decode_x_or_z(z_syndrome)
    return np.concatenate([x_correction, z_correction], axis=0)

In [75]:
correction = steane_lookup_decoder(syndrome)
print(binary_symplectic_to_pauli(correction))

[0, 0, 0, 0, 0, 0, 1]


In [76]:
# Our decoder should map single-qubit errors back to themselves.
for i in range(7):
    for p in [1, 2, 3]:
        err_pauli = [0] * 7
        err_pauli[i] = p
        err_binary = pauli_to_binary_symplectic(err_pauli)
        syndrome = error_to_steane_syndrome(err_binary)
        correction_binary = steane_lookup_decoder(syndrome)
        correction_pauli = binary_symplectic_to_pauli(correction_binary)
        assert correction_pauli == err_pauli

In [80]:
# The Steane code can't fix two-qubit errors.
err_pauli = [1, 0, 0, 1, 0, 0, 0]
err_binary = pauli_to_binary_symplectic(err_pauli)
syndrome = error_to_steane_syndrome(err_binary)
correction_binary = steane_lookup_decoder(syndrome)
correction_pauli = binary_symplectic_to_pauli(correction_binary)
print(err_pauli, syndrome, correction_pauli)
print(err_binary ^ correction_binary)

[1, 0, 0, 1, 0, 0, 0] [ True False  True False False False] [0, 0, 0, 0, 1, 0, 0]
[ True False False  True  True False False False False False False False
 False False]
